# Comparison between "Comprehensive upper-air observation network from 1905 to present" and "Insitu IGRA radiosoundings baseline network"

**Contains modified Copernicus Climate Change Service Information 2020**
under [License](https://apps.ecmwf.int/datasets/licences/copernicus/)

*Copernicus Climate Change Service (C3S) - Upper Air Data Service (2020)*

The purpose of this IPython Notebook is to find differences in both mentioned data sets and to compare them. 

    Author: U. Voggenberger
    Date: 02.2021
    Contact: ulrich.voggenberger [at] univie.ac.at
    License: C3S, 2021


In [1]:
import pandas
import numpy as np
import sys, zipfile, os, time
import matplotlib.pyplot as plt
import glob
import datetime
import urllib3
import cdsapi
import xarray
import shutil
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (16, 10),
         'axes.labelsize': 20,
         'axes.titlesize': 24,
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
pylab.rcParams.update(params)

In [3]:
def request(rqdict, source):
    t0 = time.time()

    c = cdsapi.Client()
    r = c.retrieve(
        source,rqdict)
    if True:
        r.download(target='download.zip')
        assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    z = zipfile.ZipFile('download.zip')
    z.extractall(path='./test/')
    z.close()
    print('Request took: ' + str(time.time() - t0) + ' seconds')
    
    files = glob.glob('./test/*.nc')
    
    if files[0].split('/')[-1].startswith('IGRA'):
        ds = xarray.open_dataset(files[0])            
        data = ds.to_dataframe()
        for i in files[1:]:
            ds = xarray.open_dataset(i)            
            data = data.append(ds.to_dataframe())

    else:
        ds = xarray.open_dataset(files[0]) 
        to_be_removed = ['obs', 'string14', 'trajectory', 'trajectory_index', 'trajectory_label']
        ds = ds.drop_vars(to_be_removed)
        data = ds.to_dataframe()
        for i in files[1:]:
            ds = xarray.open_dataset(i)  
            ds = ds.drop_vars(to_be_removed)
            data = data.append(ds.to_dataframe())
            
    os.remove('download.zip')
    try:
       shutil.rmtree('./test/')
    except:
       print('Error while deleting directory')
    
    return data

---

# Comparing available stations for IGRA (and its harmonized version) and Comprehensive upper-air observation network (CUON)

---

In [4]:
cuon = 'insitu-comprehensive-upper-air-observation-network'
igra = 'insitu-observations-igra-baseline-network'

## Requesting the global data and checking for the number of different stations


In [6]:
dates = ['1980-01-01/1980-01-02','1990-01-01/1990-01-02','2000-01-01/2000-01-02','2010-01-01/2010-01-02','2020-01-01/2020-01-02']   
for i in dates:
    print('date range: ' + i)
    da_c = request({
        'variable': 'temperature',
        'period': i,
    }, cuon)
    print('requestable stations from CUON: ' + str(len(da_c.drop_duplicates(['lat', 'lon']))))

    da_i = request({
        'source': 'IGRA',
        'variable': ['air_temperature'],
        'period': i,
    }, igra)
    print('requestable stations from IGRA: ' + str(len(da_i.drop_duplicates(['latitude', 'longitude']))))
    da_ih = request({
        'source': 'IGRA_H',
        'variable': ['air_temperature'],
        'period': i,
    }, igra)
    print('requestable stations from IGRA: ' + str(len(da_ih.drop_duplicates(['latitude', 'longitude']))))
    print('\n')

2021-02-03 11:50:00,110 INFO Welcome to the CDS
2021-02-03 11:50:00,111 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-02-03 11:50:00,172 INFO Request is queued


date range: 1980-01-01/1980-01-02


2021-02-03 11:50:01,211 INFO Request is running
2021-02-03 11:51:53,999 INFO Request is completed
2021-02-03 11:51:54,000 INFO Downloading http://136.156.132.176/cache-compute-0002/cache/data1/adaptor.comprehensive_upper_air.retrieve-1612349486.5454996-17181-44-3eafb52f-35c7-44a4-a05d-20a8bed753b4.zip to download.zip (20.8M)
2021-02-03 11:51:55,198 INFO Download rate 17.4M/s  


Request took: 115.91240072250366 seconds


2021-02-03 11:52:03,612 INFO Welcome to the CDS
2021-02-03 11:52:03,614 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-02-03 11:52:03,659 INFO Request is queued


...

requestable stations from CUON: 923


2021-02-03 11:54:42,915 INFO Request is running
2021-02-03 11:54:44,454 INFO Request is completed
2021-02-03 11:54:44,455 INFO Downloading http://136.156.132.176/cache-compute-0002/cache/data2/adaptor.insitu_reference.retrieve_test-1612349683.0694807-22546-20-cb5f93a8-5908-462f-a7d1-539c75c5f65d.zip to download.zip (533.8K)
2021-02-03 11:54:44,637 INFO Download rate 2.9M/s 
2021-02-03 11:54:44,783 INFO Welcome to the CDS
2021-02-03 11:54:44,784 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-02-03 11:54:44,826 INFO Request is queued


Request took: 2.864351749420166 seconds
requestable stations from IGRA: 632


2021-02-03 11:54:45,864 INFO Request is running
2021-02-03 11:54:47,405 INFO Request is completed
2021-02-03 11:54:47,406 INFO Downloading http://136.156.132.176/cache-compute-0000/cache/data1/adaptor.insitu_reference.retrieve_test-1612349685.8007288-32219-19-522867a0-3a4d-4bbc-84ee-461b6ec7228a.zip to download.zip (397.4K)
2021-02-03 11:54:47,583 INFO Download rate 2.2M/s 


Request took: 2.8553895950317383 seconds
requestable stations from IGRA: 454


